# Self-Driving Car Engineer Nanodegree


## Project: Build a Traffic Light Detector for CarND-Capstone project

In this notebook, we are building a Traffic Light Detector for the Capstone project for the Udacity Self Driving Cars nanodegree. 

In addition to implementing code, there is a [write up file](https://github.com/edupaz2/Traffic-Light-Classifier-Project/blob/master/writeup.md).

## Step 0: Prepare the Data

In [3]:
import cv2

def img2png(jpgs_list, remove=True):
    for j in jpgs_list:
        # Convert to PNG
        img = cv2.imread(j)
        print(j[:-3] + 'png')
        cv2.imwrite(j[:-3] + 'png', img)
        # Delete the file
        if remove:
            os.remove(j)

def img2jpg(jpgs_list, remove=True):
    for j in jpgs_list:
        # Convert to PNG
        img = cv2.imread(j)
        print(j[:-3] + 'jpg')
        cv2.imwrite(j[:-3] + 'jpg', img)
        # Delete the file
        if remove:
            os.remove(j)

In [6]:
img2jpg(['../0.Capstone/shared/img_292_5.png'], remove=False)

../0.Capstone/shared/img_292_5.jpg


In [ ]:
# Transform JPG to PNG
transformJPG2PNG = False
if transformJPG2PNG:
    l = glob.glob('./sim-data/**/*.jpg', recursive=True)
    img2png(l)


print("Done")


In [ ]:
# Rename files Consecutively

def renameConsecutively(path):
    counter = 1
    filelist = os.listdir(path)
    zerofill = len(str(abs(len(filelist))))
    for f in filelist:
        oldname = '{0}/{1}'.format(path, f)
        newname = '{0}/{1}.png'.format(path, str(counter).zfill(zerofill))
        os.rename(oldname, newname)
        counter += 1

rename = False
if rename:
    renameConsecutively('sim-data/green')
    renameConsecutively('sim-data/none')
    renameConsecutively('sim-data/red')
    renameConsecutively('sim-data/yellow')

print('Done')

## Create the pickle data
At this point we must have:
- Directory 'sim-data' containing the pictures in PNG format.
- Inside 'sim-data', 4 folders: Green, None, Red, Yellow.

In [99]:
from PIL import Image
import numpy as np
import os
import glob
import pickle

features = []
labels = []
size = 128, 128

images  = glob.glob('./sim-data/**/*.png', recursive=True)

for image_file in images:
    image = Image.open(image_file)
    image.thumbnail(size, Image.ANTIALIAS)
    #image.load()
    # Load image data as 1 dimensional array
    # We're using float32 to save on memory space
    feature = np.array(image, dtype=np.float32)#.flatten()
    labelStr = os.path.split(os.path.split(image_file)[0])[1]
    if labelStr == 'green':
        labels.append(1)
    elif labelStr == 'red':
        labels.append(2)
    elif labelStr == 'yellow':
        labels.append(3)
    else:
        labels.append(0)

    features.append(feature)

# Save the data for easy access
pickle_file = 'sim-data/sim-data.pickle'
print('Saving data to pickle file...')
try:
    with open(pickle_file, 'wb+') as pfile:
        pickle.dump(
            {
                'features': features,
                'labels': labels,
            },
            pfile, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

# Free up memory
del pickle_file
del features
del labels
print('Done')

Saving data to pickle file...
Done


## Verify the data


In [100]:
# Reload the data
pickle_file = 'sim-data/sim-data.pickle'
with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    features = pickle_data['features']
    labels = pickle_data['labels']
    del pickle_data  # Free up memory

# Number of features
n_features = len(features)

# Number of labels
n_labels = len(labels)

# What's the shape of an traffic sign image?
image_shape = features[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(labels))

print('Features: {0}, Labels: {1}, Image_Shape: {2}, Classes: {3}'.format(n_features, n_labels, image_shape, n_classes))

del pickle_file
del features
del labels

Features: 1121, Labels: 1121, Image_Shape: (96, 128, 3), Classes: 4


## Augment the data

In [1]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
import random
import cv2
import numpy as np
import pickle

class ImagePreprocessor():
    
    toGrayMask = np.array((0.299, 0.587, 0.114))
    
    def __init__(self):
        pass
    
    def preprocess(self, image_data):
        # GrayScale and Normalize the input
        return self.normalize_grayscale(self.image_to_gray(image_data))

    def image_to_gray(self, image_data):
        gray = np.dot(image_data[...,:3], self.toGrayMask)
        gray = gray.squeeze()
        gray = gray.reshape(32, 32, 1)
        return gray

    def normalize_grayscale(self, image_data):
        return (image_data-128)/128

    # Functions extracted from cv2 doc
    # http://opencv-python-tutroals.readthedocs.io/en/latest/ \
    # py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html
    def translate(self, image_data):
        rows,cols = image_data.shape[0], image_data.shape[1]
        x_trans, y_trans = np.random.uniform(-2, 2), np.random.uniform(-2, 2)
        M = np.float32([[1, 0, x_trans],[0, 1, y_trans]])
        return cv2.warpAffine(image_data,M,(cols,rows))

    def rotate(self, image_data):
        rows,cols = image_data.shape[0], image_data.shape[1]
        rotate_factor = np.random.uniform(-15, 15)
        M = cv2.getRotationMatrix2D((cols/2,rows/2), rotate_factor, 1)
        return cv2.warpAffine(image_data,M,(cols,rows))

    def perspective(self, image_data):
        rows,cols = image_data.shape[0], image_data.shape[1]
        fctr = np.random.uniform(0.05*rows, 0.2*rows)
        pts1 = np.float32([[fctr,fctr],[rows-fctr,fctr],[fctr,cols-fctr],[rows-fctr,cols-fctr]])
        pts2 = np.float32([[0,0],[rows,0],[0,cols],[rows,cols]])
        M = cv2.getPerspectiveTransform(pts1,pts2)
        return cv2.warpPerspective(image_data, M, (cols, rows))
    
    def affine(self, image_data):
        rows,cols = image_data.shape[0], image_data.shape[1]
        rnd1x, rnd1y = np.random.uniform(0, rows*0.25), np.random.uniform(0, cols*0.25)
        rnd2x, rnd2y = np.random.uniform(rows*0.75, rows), np.random.uniform(0, cols*0.35)
        rnd3x, rnd3y = np.random.uniform(0, rows*0.35), np.random.uniform(cols*0.75, cols)
        pts1 = np.float32([[0,0],[rows-1,0],[0,cols-1]])
        pts2 = np.float32([[rnd1x, rnd1y],[rnd2x, rnd2y],[rnd3x, rnd3y]])
        M = cv2.getAffineTransform(pts1,pts2)
        return cv2.warpAffine(image_data, M, (cols,rows))
    
    def research(self, image_data):
        # Define a kernel size and apply Gaussian smoothing
        #kernel_size = 5
        #return cv2.GaussianBlur(image_data.squeeze(),(kernel_size, kernel_size),0)
        
        #image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
        #return cv2.adaptiveThreshold(image_data,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        
        return cv2.bitwise_not(image_data)

    def random_effect(self, image_data):
        # Apply just one effect at a time
        effects = [self.translate, self.rotate, self.perspective, self.affine, self.research]
        effects_p = [0.2, 0.2, 0.25, 0.25, 0.1]
        effects_count = random.randint(2, len(effects)) # at least 2 effects
        effects_toapply = np.random.choice(len(effects), effects_count, replace=False, p=effects_p)
        for idx in effects_toapply:
            image_data = effects[idx](image_data)

        return image_data

print('1. Loading Pickle...')
# Reload the data
pickle_file = 'sim-data/sim-data.pickle'
with open(pickle_file, 'rb') as f:
    pickle_data = pickle.load(f)
    features = pickle_data['features']
    labels = pickle_data['labels']
    del pickle_data  # Free up memory

# Number of features
n_features = len(features)

# Number of labels
n_labels = len(labels)

# What's the shape of an traffic sign image?
image_shape = features[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(labels))

print('Features: {0}, Labels: {1}, Image_Shape: {2}, Classes: {3}'.format(n_features, n_labels, image_shape, n_classes))

print('Done')
print('2. Augmenting data...')
### Let´s add some random data to make our training data bigger
# Find out the class with more items, and increase all classes 35%
from collections import defaultdict
from sklearn.utils import shuffle
train_data_idxs = defaultdict(list)
for i in range(len(features)):
    train_data_idxs[labels[i]].append(i)

features_aug_l = []
labels_aug_l = []
preprocessor = ImagePreprocessor()

hist, bin_edges = np.histogram(labels, bins=n_classes)
increase_to = int(np.amax(hist)*1.5)
for labelidx in range(n_classes):
    # Iterate in each class
    #to_generate = random.randint(hist[labelidx], increase_to)
    to_generate = increase_to - hist[labelidx]
    for j in range(to_generate):
        # Pick a random image of the same class
        rnd_idx = np.random.choice(train_data_idxs[labelidx])
        rnd_img = features[rnd_idx]
        # Do a random effect
        image = preprocessor.random_effect(rnd_img)
        # Add it to training data
        features_aug_l.append(image)
        labels_aug_l.append(labelidx)
        
# Everyday Im shuffling
features_aug_l, labels_aug_l = shuffle(features_aug_l, labels_aug_l)
print('Done')

print('3. Validating data...')
print('Feature: {0}'.format(len(features)))
print('Feature shape: {0}'.format(features[0].shape))
print('-------------')

features_aug = np.append(features, features_aug_l, axis=0)
del features
del features_aug_l

labels_aug = np.append(labels, labels_aug_l)
del labels
del labels_aug_l

print('Augmented features: {0}'.format(features_aug.shape))
print('Augmented features shape: {0}'.format(features_aug[0].shape))
print('Done')

### Then save it so we can start from there later
# Save the data for easy access

pickle_file = 'sim-data/sim-data-augmented.pickle'
print('4. Saving data to pickle file...')
try:
    with open(pickle_file, 'wb+') as pfile:
        pickle.dump(
            {
                'features': features_aug,
                'labels': labels_aug,
            },
            pfile, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

print('Done')

1. Loading Pickle...
Features: 1121, Labels: 1121, Image_Shape: (96, 128, 3), Classes: 4
Done
2. Augmenting data...


/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Done
3. Validating data...
Feature: 1121
Feature shape: (96, 128, 3)
-------------
Augmented features: (3400, 96, 128, 3)
Augmented features shape: (96, 128, 3)
Done
4. Saving data to pickle file...
Done
